# Estimators

### Estimating context trees using CTM/BIC

In [1]:
import sys
sys.path.insert(0,'../..')
from g4l.estimators.bic import BIC
from g4l.data import Sample
from g4l.bootstrap.resampling import BlockResampling
from g4l.bootstrap import Bootstrap
import numpy as np

In [2]:
# 1. Create a Sample object
sample_file = 'resources/samples/model1_5000_s001.txt'
X = Sample(sample_file, [0, 1])
print('sample loaded:', X.data[0:40], '...')

sample loaded: 1001010100101010101010100101010010101001 ...


In [6]:
penalty_constant = 0.08
max_depth = 6

# 2. Instantiate the estimator
bic = BIC(penalty_constant, max_depth)

# 3. Estimate model using the loaded sample
bic.fit(X)

# 4. Collect the estimated model
tree = bic.context_tree
# TODO: adicionar parâmetro para modificar visualização da string
print('Nodes:', tree.to_str())
print('Log-likelihood:', round(tree.log_likelihood(), 4))
print('# of contexts:', tree.num_contexts())

Nodes: 000 1 10 100
Log-likelihood: -1658.0061
# of contexts: 4


In [7]:
# Complete information regarding nodes are available in the returned object

# Listing contexts (Pandas DataFrame object):
tree.tree()

,node_idx,depth_idx,node,freq,active,depth,parent_idx,likelihood,active_children,num_child_nodes,likelihood_pen,v_node,v_node_sum,indicator,comp_aux
10,10,10,000,147,1,3,4,-79.502509,0.0,2.0,-80.427664,-80.427664,-80.880319,0,False
1,1,1,1,2114,1,1,0,0.000000,0.0,1.0,-0.925155,-0.925155,-0.925155,0,False
3,3,3,10,2113,1,2,2,-1283.125824,0.0,1.0,-1284.050978,-1282.832302,-1282.832302,1,False
6,6,6,100,625,1,3,4,-295.377750,0.0,1.0,-296.302904,-296.103301,-296.103301,1,False


In [9]:
# Transition probabilities associated to the nodes
tree.transition_probs.head(10)

,next_symbol,freq,prob,likelihood
idx,,,,
10,0,34,0.231293,-49.778450
10,1,113,0.768707,-29.724059
1,0,2114,1.000000,0.000000
3,0,625,0.295788,-761.320230
3,1,1488,0.704212,-521.805594
6,0,113,0.180800,-193.271113
6,1,512,0.819200,-102.106637
